In [1]:


import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/10/27 20:02:49 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/10/27 20:02:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b0bf38be-9199-47bd-a4b4-815108775cff;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.1.0 in central
	found com.typesafe#config;1.3.0 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google

Spark NLP version 3.1.0
Apache Spark version: 3.0.2


In [2]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [3]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("pos"),
        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


+--------+---+---------+
|   token|pos|ner_label|
+--------+---+---------+
|       i|  0|        0|
|    want|  0|      Rqt|
|      to|  0|        0|
|      go|  0|      Act|
|      to|  0|        0|
|     the|  0|        0|
|location|  0|      Loc|
|       i|  0|        0|
|    want|  0|      Rqt|
|      to|  0|        0|
|      go|  0|      Act|
|      to|  0|        0|
|     the|  0|        0|
| kitchen|  0|      Loc|
|       i|  0|        0|
|    want|  0|      Rqt|
|      to|  0|        0|
|      go|  0|      Act|
|      to|  0|        0|
|     the|  0|        0|
+--------+---+---------+
only showing top 20 rows



In [4]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [5]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ / ]Download done! Loading the resource.
[OK!]


In [6]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-10-27 20:03:03.335566: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 20:03:03.380749: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_nam

Training started - total epochs: 500 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 22 - training examples: 90
Epoch 1/500 started, lr: 0.003, dataset size: 90
Epoch 1/500 - 0.68s - loss: 32.42958 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.17s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 0	 11	 0.0	 0.0	 0.0
Loc	 5	 0	 18	 1.0	 0.2173913	 0.35714287
Act	 16	 0	 12	 1.0	 0.5714286	 0.72727275
0	 64	 44	 0	 0.5925926	 1.0	 0.74418604
Obj	 0	 0	 2	 0.0	 0.0	 0.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 96 fp: 44 fn: 44 labels: 7
Macro-average	 prec: 0.5132275, rec: 0.39840284, f1: 0.44858378
Micro-average	 prec: 0.6857143, rec: 0.6857143, f1: 0.6857143
Epoch 2/500 started, lr: 0.0029850747, dataset size: 90
Epoch 2/500 - 0.09s - loss: 24.08797 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 0	 11	 0.0	 

Epoch 15/500 - 0.08s - loss: 0.20682809 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 139 fp: 1 fn: 1 labels: 8
Macro-average	 prec: 0.75, rec: 0.75, f1: 0.75
Micro-average	 prec: 0.99285716, rec: 0.99285716, f1: 0.99285716
Epoch 16/500 started, lr: 0.0027906976, dataset size: 90
Epoch 16/500 - 0.08s - loss: 0.17858659 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigge

Epoch 30/500 - 0.08s - loss: 0.04661304 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 139 fp: 1 fn: 1 labels: 8
Macro-average	 prec: 0.75, rec: 0.75, f1: 0.75
Micro-average	 prec: 0.99285716, rec: 0.99285716, f1: 0.99285716
Epoch 31/500 started, lr: 0.0026086958, dataset size: 90
Epoch 31/500 - 0.08s - loss: 0.05545099 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigge

Epoch 45/500 - 0.08s - loss: 0.024470093 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 139 fp: 1 fn: 1 labels: 8
Macro-average	 prec: 0.75, rec: 0.75, f1: 0.75
Micro-average	 prec: 0.99285716, rec: 0.99285716, f1: 0.99285716
Epoch 46/500 started, lr: 0.0024489795, dataset size: 90
Epoch 46/500 - 0.08s - loss: 0.012510774 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trig

...

Epoch 465/500 - 0.08s - loss: 3.2341862E-4 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 139 fp: 1 fn: 1 labels: 8
Macro-average	 prec: 0.75, rec: 0.75, f1: 0.75
Micro-average	 prec: 0.99285716, rec: 0.99285716, f1: 0.99285716
Epoch 466/500 started, lr: 9.0225565E-4, dataset size: 90
Epoch 466/500 - 0.08s - loss: 1.3489199E-4 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0

Epoch 480/500 - 0.08s - loss: 7.3094503E-4 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 139 fp: 1 fn: 1 labels: 8
Macro-average	 prec: 0.75, rec: 0.75, f1: 0.75
Micro-average	 prec: 0.99285716, rec: 0.99285716, f1: 0.99285716
Epoch 481/500 started, lr: 8.82353E-4, dataset size: 90
Epoch 481/500 - 0.07s - loss: 1.2890516E-4 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
T

Epoch 495/500 - 0.08s - loss: 0.0015140856 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 11	 0	 0	 1.0	 1.0	 1.0
tp: 139 fp: 1 fn: 1 labels: 8
Macro-average	 prec: 0.75, rec: 0.75, f1: 0.75
Micro-average	 prec: 0.99285716, rec: 0.99285716, f1: 0.99285716
Epoch 496/500 started, lr: 8.6330937E-4, dataset size: 90
Epoch 496/500 - 0.07s - loss: 0.0021869824 - batches: 4
Quality on validation dataset (20.0%), validation examples = 18
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 11	 0	 0	 1.0	 1.0	 1.0
Loc	 23	 0	 0	 1.0	 1.0	 1.0
Act	 28	 0	 0	 1.0	 1.0	 1.0
Neg	 0	 1	 0	 0.0	 0.0	 0.0
0	 64	 0	 0	 1.0	 1.0	 1.0
Obj	 2	 0	 0	 1.0	 1.0	 1.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0

2022-10-27 20:03:48.610445: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

CPU times: user 456 ms, sys: 148 ms, total: 604 ms
Wall time: 49.6 s


In [7]:
import pyspark.sql.functions as F
predictions = ner_model.transform(testing_data) 
predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
           .select(F.expr("cols['0']").alias("token"),
                   F.expr("cols['1']").alias("ground_truth"),
                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

+--------+------------+----------+
|token   |ground_truth|prediction|
+--------+------------+----------+
|i       |0           |0         |
|want    |Rqt         |Rqt       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|location|Loc         |Loc       |
|i       |0           |0         |
|want    |Rqt         |Rqt       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
|kitchen |Loc         |Loc       |
|i       |0           |0         |
|want    |Rqt         |Rqt       |
|to      |0           |0         |
|go      |Act         |Act       |
|to      |0           |0         |
|the     |0           |0         |
+--------+------------+----------+
only showing top 20 rows



In [8]:
from sklearn.metrics import classification_report
 
preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()
 
print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131
         Act       1.00      1.00      1.00        50
         Loc       1.00      1.00      1.00        42
         Obj       1.00      1.00      1.00         3
         Rqt       1.00      1.00      1.00        27
     Trigger       1.00      1.00      1.00         3

    accuracy                           1.00       256
   macro avg       1.00      1.00      1.00       256
weighted avg       1.00      1.00      1.00       256



In [9]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [10]:
text_list = [
    """Navagate to kitchen"""
]


In [11]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


2022-10-27 20:03:52.724232: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1139] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [12]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_chunk': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'embeddings': ['navigate', 'to', 'kitchen']}
['Act', '0', 'Loc']


In [13]:
light.annotate("Navigate to kitchen")


{'document': ['Navigate to kitchen'],
 'ner_chunk': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'embeddings': ['Navigate', 'to', 'kitchen']}

In [14]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    print(light.annotate(text))
    text = input("Enter New Text\n")

Enter Testing Text
i want to go to the kitchen
{'document': ['i want to go to the kitchen'], 'ner_chunk': ['want to go to the kitchen'], 'token': ['i', 'want', 'to', 'go', 'to', 'the', 'kitchen'], 'ner': ['0', 'Rqt', '0', 'Act', '0', '0', 'Loc'], 'embeddings': ['i', 'want', 'to', 'go', 'to', 'the', 'kitchen']}
Enter New Text
exit


In [15]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [16]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




